In [5]:
from stanza.server import CoreNLPClient
import pickle
import pandas as pd
import spacy
from tqdm import tqdm
import requests
import numpy as np

In [2]:
nlp = spacy.load('en_core_web_lg')
var = pickle.load(open('./cleaned_data_split/a0.sav', 'rb'))
doc=nlp(var[0])
sent=str(list(doc.sents)[0])
sent = sent.rstrip()

In [3]:
result=[]
url = 'http://localhost:8080/minie/query'
r = requests.post(url, data=sent)
for facts in r.json()['facts']:
    result.append({'subject':facts['subject'],'relation': facts['predicate'],'object':facts['object']})

In [4]:
result2=[]
with CoreNLPClient(be_quite=False, annotators=["openie"], memory='16G', endpoint='http://localhost:1234', threads=12) as client:
    ann = client.annotate(sent,output_format='json')
    for sentence in ann['sentences']:
        for triple in sentence['openie']:
            if triple:
                result2.append({'subject':triple['subject'],'relation': triple["relation"],'object':triple['object']})

2021-10-07 16:01:26 INFO: Writing properties to tmp file: corenlp_server-dff47d6fbf4a4df4.props
2021-10-07 16:01:26 INFO: Starting server with command: java -Xmx16G -cp C:\Users\pross\stanza_corenlp\* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 1234 -timeout 60000 -threads 12 -maxCharLength 100000 -quiet False -serverProperties corenlp_server-dff47d6fbf4a4df4.props -annotators openie -preload -outputFormat serialized


In [7]:
sent

'objective: this retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven mycoplasma pneumoniae infections at king abdulaziz university hospital, jeddah, saudi arabia.'

In [8]:
np.array(result) #minie

array([{'subject': 'retrospective chart review', 'relation': 'describes', 'object': 'epidemiology of QUANT_O_1 patients with culture-proven mycoplasma pneumoniae infections at king abdulaziz university hospital'},
       {'subject': 'retrospective chart review', 'relation': 'describes', 'object': 'clinical features'},
       {'subject': 'retrospective chart review', 'relation': 'describes', 'object': 'epidemiology of QUANT_O_1 patients with culture-proven mycoplasma pneumoniae infections at jeddah'},
       {'subject': 'king abdulaziz university hospital', 'relation': 'is', 'object': 'saudi arabia'},
       {'subject': 'jeddah', 'relation': 'is', 'object': 'saudi arabia'}],
      dtype=object)

In [9]:
np.array(result2) #openie

array([{'subject': 'retrospective chart review', 'relation': 'epidemiology of', 'object': '40 patients with culture proven mycoplasma pneumoniae infections at king abdulaziz university hospital'},
       {'subject': 'chart review', 'relation': 'epidemiology of', 'object': '40 patients with mycoplasma pneumoniae infections at king university hospital'},
       {'subject': 'retrospective chart review', 'relation': 'describes', 'object': 'epidemiology of 40 patients with mycoplasma pneumoniae infections'},
       {'subject': 'retrospective chart review', 'relation': 'describes', 'object': 'epidemiology of 40 patients with culture proven mycoplasma pneumoniae infections'},
       {'subject': 'retrospective chart review', 'relation': 'epidemiology of', 'object': '40 patients with proven mycoplasma pneumoniae infections'},
       {'subject': 'retrospective chart review', 'relation': 'describes', 'object': 'epidemiology of 40 patients with mycoplasma pneumoniae infections at king university h